# IBM Data Science Capstone Project Notebook

## Project Title: Alternative areas to Asheville, NC

### Introduction / Business Problem

Asheville, North Carolina has been a tourist destination since the middle of the 19th century when people came to the area seeking relief from tuberculosis. At the time it was thought the climate promoted healing and relief of tuberculosis and numerous boarding houses and sanitariums were established to support this industry. In more recent years, Asheville has seen double-digit population growth since the 1990s and is commonly featured on lists of top destinations. Because of this continued growth and demand the cost of housing in Asheville is higher than most other areas in North Carolina. 

The objective of this project is to compare Asheville with other metropolitan areas in North Carolina to identify similar areas in terms of businesses and venues but with lower cost housing. There are factors that we are not considering that can be considered unique to Asheville which could ultimately influence a buyer's decision such as the geography and climate.

The audience for our project is people who would like to purchase a single-family home in Asheville but either can’t afford the cost or choose not to pay the prices. We are attempting to help these people find alternative locations in North Carolina that are similar in terms of businesses/venues to Asheville but with lower cost housing.

### Data

#### House Price Data

We will use the Federal Housing Finance Agency’s House Price Index (HPI) to determine how the price of housing compares across US Census Bureau Metropolitan Statistical Areas (MSAs). We’ll use the most recent HPI data available which is from the first quarter of 2021. We’ll also use the “all-transactions” type which includes both refinance mortgages and purchase-only data. We’re choosing this type because it appears to be the only type for which recent data is available that includes Asheville, NC. We’ll use the non-seasonally adjusted index since it appears the seasonally adjusted index is not available for the “all-transactions” type.

https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index.aspx

NOTE: the HPI focuses on single-family house prices and primarily on mortgages that are purchased and/or securitized by Fannie Mae or Freddie Mac.

There are several MSAs listed for North Carolina, but we will be focusing our comparison on the following MSAs: Asheville, Charlotte-Concord-Gastonia, Durham-Chapel Hill, Greensboro-High Point, Hickory-Lenoir-Morganton, Raleigh-Cary, Wilmington, and Winston-Salem.

#### Location / Venue Data

We will use the Foursquare API to acquire location/venue data by zip-code for comparison.

https://developer.foursquare.com/docs/places-api/

We will use the US Department of Housing and Urban Development’s (HUD) crosswalk between MSAs and zip-codes to acquire the list of zip-codes within each MSA. NOTE: zip-codes can sometimes be located partially inside and outside of an MSA.

https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data


### Load and Clean Data

#### Install necessary libraries and set options

In [1]:
# install necessary libraries and set options
import os, types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
import requests
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Install complete')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 9.9 MB/s  eta 0:00:01
Install complete


In [40]:
!pip install folium
import folium
print('Install complete')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.9 MB/s  eta 0:00:01
Install complete


#### Load price index and zip-code data

In [2]:
# The code was removed by Watson Studio for sharing.

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa
0,traditional,all-transactions,quarterly,MSA,"Asheville, NC",11700,2021,1,332.44
1,traditional,all-transactions,quarterly,MSA,"Charlotte-Concord-Gastonia, NC-SC",16740,2021,1,257.81
2,traditional,all-transactions,quarterly,MSA,"Durham-Chapel Hill, NC",20500,2021,1,246.64
3,traditional,all-transactions,quarterly,MSA,"Greensboro-High Point, NC",24660,2021,1,190.91
4,traditional,all-transactions,quarterly,MSA,"Hickory-Lenoir-Morganton, NC",25860,2021,1,217.67


In [3]:
HPI.shape

(8, 9)

In [23]:
# clean up HPI dataframe
HPInew = HPI.drop(['hpi_type','hpi_flavor','frequency','level','yr','period'], axis = 1, errors = 'ignore')
HPInew.rename(columns = {'place_name':'MSA_NAME', 'place_id':'MSA_ID', 'index_nsa':'PRICE_INDEX'}, inplace = True)
HPInew.head()

,MSA_NAME,MSA_ID,PRICE_INDEX
0,"Asheville, NC",11700,332.44
1,"Charlotte-Concord-Gastonia, NC-SC",16740,257.81
2,"Durham-Chapel Hill, NC",20500,246.64
3,"Greensboro-High Point, NC",24660,190.91
4,"Hickory-Lenoir-Morganton, NC",25860,217.67


In [4]:
# The code was removed by Watson Studio for sharing.

,ZIP,CBSA,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE
0,27006,49180,ADVANCE,NC
1,27009,49180,BELEWS CREEK,NC
2,27010,49180,BETHANIA,NC
3,27011,49180,BOONVILLE,NC
4,27012,49180,CLEMMONS,NC


In [5]:
HUD.shape

(588, 4)

In [25]:
# clean up HUD dataframe
HUDnew = HUD.drop(['USPS_ZIP_PREF_STATE'], axis = 1, errors = 'ignore')
HUDnew.rename(columns = {'USPS_ZIP_PREF_CITY':'CITY'}, inplace = True)
HUDnew.head()

,ZIP,CBSA,CITY
0,27006,49180,ADVANCE
1,27009,49180,BELEWS CREEK
2,27010,49180,BETHANIA
3,27011,49180,BOONVILLE
4,27012,49180,CLEMMONS


In [30]:
# merge HPInew and HUDnew dataframes on MSA
HPIandHUD = HPInew.merge(HUDnew, how = 'left', left_on = 'MSA_ID', right_on = 'CBSA')
HPIandHUD.drop('CBSA', axis = 1, inplace = True, errors = 'ignore')
HPIandHUD.reset_index()
HPIandHUD.head()

,MSA_NAME,MSA_ID,PRICE_INDEX,ZIP,CITY
0,"Asheville, NC",11700,332.44,28655,MORGANTON
1,"Asheville, NC",11700,332.44,28701,ALEXANDER
2,"Asheville, NC",11700,332.44,28704,ARDEN
3,"Asheville, NC",11700,332.44,28709,BARNARDSVILLE
4,"Asheville, NC",11700,332.44,28710,BAT CAVE


In [31]:
HPIandHUD.shape

(588, 5)

### Get latitude and longitude data for each zip code

In [9]:
latitude=[]
longitude=[]
for code in HPIandHUD['ZIP']:
    g = geocoder.arcgis('{}'.format(code))
    # print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}'.format(code))
        # print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

# create dataframes for the latitude and longitude lists
latitude_df = pd.DataFrame(latitude, columns = {'Latitude'})
longitude_df = pd.DataFrame(longitude, columns = {'Longitude'})

# merge the latitude and longitude dataframes with the neighborhood dataframe
lat_long_df = latitude_df.merge(longitude_df, how = 'left', left_index = True, right_index = True)
HPIandHUD = HPIandHUD.merge(lat_long_df, how = 'left', left_index = True, right_index = True)
HPIandHUD.head()

,MSA_NAME,MSA_ID,PRICE_INDEX,ZIP,CITY,Latitude,Longitude
0,"Asheville, NC",11700,332.44,28655,MORGANTON,35.744745,-81.685285
1,"Asheville, NC",11700,332.44,28701,ALEXANDER,35.647905,-82.638820
2,"Asheville, NC",11700,332.44,28704,ARDEN,35.476592,-82.516678
3,"Asheville, NC",11700,332.44,28709,BARNARDSVILLE,35.780054,-82.461256
4,"Asheville, NC",11700,332.44,28710,BAT CAVE,35.452780,-82.289310


### Explore and Cluster MSAs

#### Define Foursquare credentials

In [10]:
# The code was removed by Watson Studio for sharing.

#### Create a function to look up venues from Foursquare for each lat and long pair

In [11]:
# create a function to get nearby venues for each MSA and lat/long pair
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['MSA_NAME', 
                  'MSA Latitude', 
                  'MSA Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# run the above function on each MSA and lat/long combination and create a new dataframe called nc_venues
nc_venues = getNearbyVenues(names=HPIandHUD['MSA_NAME'],
                                   latitudes=HPIandHUD['Latitude'],
                                   longitudes=HPIandHUD['Longitude']
                                  )

In [14]:
nc_venues.head()

,MSA_NAME,MSA Latitude,MSA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Asheville, NC",35.744745,-81.685285,Catawba Valley Brewing Co.,35.744663,-81.685645,Brewery
1,"Asheville, NC",35.744745,-81.685285,Root And Vine Restaurant,35.744705,-81.688970,American Restaurant
2,"Asheville, NC",35.744745,-81.685285,Grind Cafe,35.744748,-81.689171,Coffee Shop
3,"Asheville, NC",35.744745,-81.685285,Brown Mountain Bottleworks,35.746094,-81.687756,Beer Store
4,"Asheville, NC",35.744745,-81.685285,CVS pharmacy,35.747544,-81.688697,Pharmacy


In [15]:
nc_venues.shape

(6206, 7)

In [17]:
# analyze each MSA
# one hot encoding
nc_onehot = pd.get_dummies(nc_venues[['Venue Category']], prefix="", prefix_sep="")

# add MSA column back to dataframe
nc_onehot.insert(0, 'MSA_NAME', nc_venues['MSA_NAME'])

nc_onehot.head()

,MSA_NAME,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Communications Building,College Gym,College Library,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Service,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residence,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South Indian Restau

#### Group rows by MSA and by taking the mean of the frequency of occurrence of each category

In [18]:
# group rows by MSA and by taking the mean of the frequency of occurrence of each category
MSA_grouped = nc_onehot.groupby(['MSA_NAME']).mean().reset_index()
MSA_grouped.head()

,MSA_NAME,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Communications Building,College Gym,College Library,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Service,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residence,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South Indian Restau

#### Define a function to sort the venues in descending order

In [19]:
# define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues for each neighborhood

In [20]:
# create the new dataframe and display the top 10 venues for each MSA
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['MSA_NAME']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
MSA_venues_sorted = pd.DataFrame(columns=columns)
MSA_venues_sorted['MSA_NAME'] = MSA_grouped['MSA_NAME']

for ind in np.arange(MSA_grouped.shape[0]):
    MSA_venues_sorted.iloc[ind, 1:] = return_most_common_venues(MSA_grouped.iloc[ind, :], num_top_venues)

MSA_venues_sorted.head()

,MSA_NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Asheville, NC",American Restaurant,Brewery,Pizza Place,Coffee Shop,Mexican Restaurant,Hotel,Fast Food Restaurant,Sandwich Place,Pharmacy,Café
1,"Charlotte-Concord-Gastonia, NC-SC",American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Coffee Shop,Hotel,Discount Store,Italian Restaurant,Restaurant
2,"Durham-Chapel Hill, NC",Pizza Place,Coffee Shop,Sandwich Place,Discount Store,American Restaurant,Mexican Restaurant,Bar,Pharmacy,Ice Cream Shop,Fast Food Restaurant
3,"Greensboro-High Point, NC",Fast Food Restaurant,American Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bar,Pizza Place,Discount Store,Burger Joint,Gas Station
4,"Hickory-Lenoir-Morganton, NC",American Restaurant,Fast Food Restaurant,Pizza Place,Pharmacy,Post Office,Coffee Shop,Restaurant,Discount Store,Burger Joint,Sandwich Place


#### Cluster MSAs by running k-means to cluster the MSAs into 2 clusters

In [34]:
# set number of clusters
kclusters = 2

MSA_grouped_clustering = MSA_grouped.drop('MSA_NAME', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(MSA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:8]

array([1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each MSA

In [35]:
# add clustering labels
MSA_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

HPIfinal = HPInew

# merge neighborhood data with venue data to add latitude/longitude for each neighborhood
HPIfinal = HPIfinal.join(MSA_venues_sorted.set_index('MSA_NAME'), on='MSA_NAME')

# drop rows that contain NaN values and convert Cluster Labels back to integers
HPIfinal.dropna(axis = 0, inplace = True)
HPIfinal = HPIfinal.astype({'Cluster Labels': 'int'})

HPIfinal.head()

,MSA_NAME,MSA_ID,PRICE_INDEX,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Asheville, NC",11700,332.44,1,American Restaurant,Brewery,Pizza Place,Coffee Shop,Mexican Restaurant,Hotel,Fast Food Restaurant,Sandwich Place,Pharmacy,Café
1,"Charlotte-Concord-Gastonia, NC-SC",16740,257.81,1,American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Coffee Shop,Hotel,Discount Store,Italian Restaurant,Restaurant
2,"Durham-Chapel Hill, NC",20500,246.64,1,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,American Restaurant,Mexican Restaurant,Bar,Pharmacy,Ice Cream Shop,Fast Food Restaurant
3,"Greensboro-High Point, NC",24660,190.91,1,Fast Food Restaurant,American Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bar,Pizza Place,Discount Store,Burger Joint,Gas Station
4,"Hickory-Lenoir-Morganton, NC",25860,217.67,0,American Restaurant,Fast Food Restaurant,Pizza Place,Pharmacy,Post Office,Coffee Shop,Restaurant,Discount Store,Burger Joint,Sandwich Place


#### Visualize the resulting clusters

In [37]:
# insert an approx central lat and long for each MSA in order to map
HPIfinal['MSA_lat'] = ['35.59009','35.22290','35.94815','36.00618','35.75282','35.81881','34.22192','36.11802']
HPIfinal['MSA_long'] = ['-82.55824','-80.84520','-78.95553','-79.87207','-81.53405','-78.71404','-77.87040','-80.20037']
HPIfinal.head()

,MSA_NAME,MSA_ID,PRICE_INDEX,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,MSA_lat,MSA_long
0,"Asheville, NC",11700,332.44,1,American Restaurant,Brewery,Pizza Place,Coffee Shop,Mexican Restaurant,Hotel,Fast Food Restaurant,Sandwich Place,Pharmacy,Café,35.59009,-82.55824
1,"Charlotte-Concord-Gastonia, NC-SC",16740,257.81,1,American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Coffee Shop,Hotel,Discount Store,Italian Restaurant,Restaurant,35.22290,-80.84520
2,"Durham-Chapel Hill, NC",20500,246.64,1,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,American Restaurant,Mexican Restaurant,Bar,Pharmacy,Ice Cream Shop,Fast Food Restaurant,35.94815,-78.95553
3,"Greensboro-High Point, NC",24660,190.91,1,Fast Food Restaurant,American Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bar,Pizza Place,Discount Store,Burger Joint,Gas Station,36.00618,-79.87207
4,"Hickory-Lenoir-Morganton, NC",25860,217.67,0,American Restaurant,Fast Food Restaurant,Pizza Place,Pharmacy,Post Office,Coffee Shop,Restaurant,Discount Store,Burger Joint,Sandwich Place,35.75282,-81.53405


In [38]:
#get lat and long for North Carolina
address = 'North Carolina, US'

geolocator = Nominatim(user_agent="nc_explorer")
location = geolocator.geocode(address)
latitude_nc = location.latitude
longitude_nc = location.longitude

In [45]:
# create map
map_clusters = folium.Map(location=[latitude_nc, longitude_nc], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, cluster in zip(HPIfinal['MSA_lat'], HPIfinal['MSA_long'], HPIfinal['MSA_NAME'], HPIfinal['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters